<a href="https://colab.research.google.com/github/ShaunakSen/Natural-Language-Processing/blob/master/Advanced_NLP_with_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Advanced NLP with spaCy

> Based on the official course: https://course.spacy.io/en

---

### Installation

In [0]:
!pip install -U spacy

In [0]:
!pip install -U spacy-lookups-data

In [0]:
!python -m spacy download en_core_web_sm

### Chapter 1: Finding words, phrases, names and concepts

#### Introduction to spaCy

**The nlp object**

At the center of spaCy is the object containing the processing pipeline. We usually call this variable "nlp".

For example, to create an English nlp object, you can import the English language class from `spacy.lang.en` and instantiate it. You can use the nlp object like a function to analyze text.

It contains all the different components in the pipeline.

It also includes language-specific rules used for tokenizing the text into words and punctuation. spaCy supports a variety of languages that are available in `spacy.lang`.



In [0]:
# Import the English language class
from spacy.lang.en import English

# Create the nlp object
nlp = English()

**The Doc Object**

When you process a text with the nlp object, spaCy creates a Doc object – short for "document". The Doc lets you access information about the text in a structured way, and no information is lost.

The Doc behaves like a normal Python sequence by the way and lets you iterate over its tokens, or get a token by its index. But more on that later!



In [6]:
# Created by processing a string of text with the nlp object

doc = nlp(text="Hello world!")

# Iterate over tokens in a Doc
for token in doc:
    print (token.text) 

Hello
world
!


**The token object**
![](https://course.spacy.io/doc.png)

Token objects represent the tokens in a document – for example, a word or a punctuation character.

Token objects also provide various attributes that let you access more information about the tokens. For example, the .text attribute returns the verbatim token text.

In [7]:
# Index into the Doc to get a single Token
token = doc[1]

# Get the token text via the .text attribute
print(token.text)

world


**The span object**

*A Span object is a slice of the document consisting of one or more tokens*. 

> It's only a view of the Doc and doesn't contain any data itself.

To create a span, you can use Python's slice notation. For example, 1:3 will create a slice starting from the token at position 1, up to – but not including! – the token at position 3.





In [8]:
span = doc[1:3]

print(span.text)

world!


**Token attributes**

Here you can see some of the available token attributes:

i is the index of the token within the parent document.

text returns the token text.

is_alpha, is_punct and like_num return boolean values indicating whether the token consists of alphabetic characters, whether it's punctuation or whether it resembles a number. For example, a token "10" – one, zero – or the word "ten" – T, E, N.

These attributes are also called lexical attributes: **they refer to the entry in the vocabulary and don't depend on the token's context**.



In [20]:
doc = nlp("It costs $5. Ten £ only")

print("Index:   ", [token.i for token in doc])
print("Text:    ", [token.text for token in doc])

print("is_alpha:", [token.is_alpha for token in doc])
print("is_punct:", [token.is_punct for token in doc])
print("like_num:", [token.like_num for token in doc])
print("is_currency:", [token.is_currency for token in doc])

Index:    [0, 1, 2, 3, 4, 5, 6, 7]
Text:     ['It', 'costs', '$', '5', '.', 'Ten', '£', 'only']
is_alpha: [True, True, False, False, False, True, False, True]
is_punct: [False, False, False, False, True, False, False, False]
like_num: [False, False, False, True, False, True, False, False]
is_currency: [False, False, True, False, False, False, True, False]
